## 0 Setup

### 0.1 Imports

In [1]:
import pandas as pd
import numpy as np

### 0.2 File paths

In [ ]:
dictionary_path = '../data/dictionary.csv'
depression_path = '../data/depression.csv'

### 0.3 Dataframes

In [3]:
dictionary = pd.read_excel(dictionary_path, skiprows=2)
depression = pd.read_csv(depression_path)

## 1 Hidden nulls
> Apparently, depression.dropna(how='all') won't drop everything we need, <br>
 because of useless classes such as "01 - can't remember" or "09 - don't know".

### 1.1 Get all the classes from the chosen columns
> Notice the NaN values in the dictionary['Descricao'] column

In [ ]:
# Charger les colonnes spécifiées dans attribute-selection.txt
with open('../data/attribute-selection.txt', 'r') as file:
    selected_columns = [line.strip() for line in file.readlines()]

# Filtrer les lignes du dictionnaire pour ne conserver que celles correspondant aux colonnes spécifiées
filtered_classes = dictionary[dictionary['Descricao'].isin(selected_columns)]

# Identifier les classes inutiles (exemple : "não sei", "não lembro")
useless_classes = filtered_classes[
    filtered_classes['Descricao'].str.contains(r'não sei|não lembro', case=False, na=False)
]

# Afficher les classes inutiles
print("Useless classes:")
print(useless_classes)

# Stocker les résultats dans un fichier CSV
useless_classes.to_csv('../data/useless_classes.csv', index=False)

In [70]:
# We can replace NaNs in that column with the previous value until a new one is found if it is not another NaN
dictionary

,ModuloParteAnotacao,PosiçãoInicial,Tamanho,Codigo,Numero,Descricao,Tipo,Categoria
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Parte 1 - Identificação e Controle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,2.0,V0001,NaN,Unidade da Federação,11,Rondônia
3,NaN,NaN,NaN,NaN,NaN,NaN,12,Acre
4,NaN,NaN,NaN,NaN,NaN,NaN,13,Amazonas
...,...,...,...,...,...,...,...,...
5216,NaN,NaN,NaN,NaN,NaN,NaN,7,"Outro método moderno (camisinha feminina, pílu..."
5217,NaN,NaN,NaN,NaN,NaN,NaN,8,Tabelinha/ Billings
5218,NaN,NaN,NaN,NaN,NaN,NaN,9,Outro método tradicional
5219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Não aplicável


#### 1.1.1 Replace NaN values with the previous value in the dictionay['Descricao'] column

In [71]:
classes = dictionary[['Descricao', 'Tipo', 'Categoria']]

# only the 'Descricao' column requires forward filling
classes['Descricao'] = classes[['Descricao']].fillna(method='ffill')
classes

/tmp/ipykernel_24697/3177588585.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  classes['Descricao'] = classes[['Descricao']].fillna(method='ffill')
/tmp/ipykernel_24697/3177588585.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classes['Descricao'] = classes[['Descricao']].fillna(method='ffill')


,Descricao,Tipo,Categoria
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,Unidade da Federação,11,Rondônia
3,Unidade da Federação,12,Acre
4,Unidade da Federação,13,Amazonas
...,...,...,...
5216,Método contraceptivo mais eficaz que faz uso,7,"Outro método moderno (camisinha feminina, pílu..."
5217,Método contraceptivo mais eficaz que faz uso,8,Tabelinha/ Billings
5218,Método contraceptivo mais eficaz que faz uso,9,Outro método tradicional
5219,Método contraceptivo mais eficaz que faz uso,NaN,Não aplicável


In [72]:
# classes where description is 'Unidade da Federação'
classes[classes['Descricao'] == 'Peso - Final (em kg)\n(3 inteiros e 1 casa decimal)']

,Descricao,Tipo,Categoria
2151,Peso - Final (em kg)\n(3 inteiros e 1 casa dec...,1 a 599,Quilogramas
2152,Peso - Final (em kg)\n(3 inteiros e 1 casa dec...,NaN,Não aplicável
5129,Peso - Final (em kg)\n(3 inteiros e 1 casa dec...,20 a 200,Quilogramas
5130,Peso - Final (em kg)\n(3 inteiros e 1 casa dec...,999,Erro
5131,Peso - Final (em kg)\n(3 inteiros e 1 casa dec...,NaN,Não aplicável


#### 1.1.2 Pick only the rows with the names of chosen columns for depression and all of their classes


In [77]:
# Note we need to remove escape sequences from the 'Descricao' column
classes['Descricao'] = classes['Descricao'].str.replace(r'\n', '', regex=True)
classes[classes['Descricao'] == 'Peso - Final (em kg)(3 inteiros e 1 casa decimal)']

/tmp/ipykernel_24697/2644128347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classes['Descricao'] = classes['Descricao'].str.replace(r'\n', '', regex=True)


,Descricao,Tipo,Categoria
2151,Peso - Final (em kg)(3 inteiros e 1 casa decimal),1 a 599,Quilogramas
2152,Peso - Final (em kg)(3 inteiros e 1 casa decimal),NaN,Não aplicável
5129,Peso - Final (em kg)(3 inteiros e 1 casa decimal),20 a 200,Quilogramas
5130,Peso - Final (em kg)(3 inteiros e 1 casa decimal),999,Erro
5131,Peso - Final (em kg)(3 inteiros e 1 casa decimal),NaN,Não aplicável


In [85]:
depression.columns.shape

# count the repeated columns
depression.columns.nunique()

55

In [87]:
# remove the repeated columns
depression.columns = pd.Series(depression.columns).str.replace(r'\.\d+', '', regex=True)
depression.columns.nunique()

52

In [74]:
depression_classes = classes[classes['Descricao'].isin(depression.columns)]

In [ ]:
# count the number of unique values in the 'Descricao' column
depression_classes['Descricao'].nunique()

30

In [76]:
depression_classes

,Descricao,Tipo,Categoria
2,Unidade da Federação,11,Rondônia
3,Unidade da Federação,12,Acre
4,Unidade da Federação,13,Amazonas
5,Unidade da Federação,14,Roraima
6,Unidade da Federação,15,Pará
...,...,...,...
4782,Quem fez isso com você,13,Outro
4783,Quem fez isso com você,NaN,Não aplicável
5129,Peso - Final (em kg)(3 inteiros e 1 casa decimal),20 a 200,Quilogramas
5130,Peso - Final (em kg)(3 inteiros e 1 casa decimal),999,Erro


In [5]:
depression.head()

,"Nas duas últimas semanas, com que frequência o(a) Sr(a) se sentiu deprimido(a), “pra baixo” ou sem perspectiva?",Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?,Algum médico já lhe receitou algum medicamento para depressão?,Nas duas últimas semanas o(a) senhor(a) usou algum medicamento para depressão?,Que idade o(a) Sr(a) tinha quando começou a fumar produto de tabaco diariamente?,O(A) Sr(a) vai ao médico/serviço de saúde regularmente por causa da depressão ou só quando tem algum problema?,Qual o principal motivo do(a) Sr(a) não visitar o médico/serviço de saúde regularmente por causa da depressão?,Por causa da depressão Faz psicoterapia,Por causa da depressão Toma medicamentos,Algum dos medicamentos para depressão foi obtido em serviço público de saúde?,...,Idade do morador na data de referência,Unidade da Federação,Tipo de área,Tipo de situação censitária,Cor ou raça,Qual é a sua orientação sexual?,Qual foi o curso mais elevado que ___frequentou,"Quem fez isso com você? (Se mais de uma pessoa, defina o pirncipal agressor)",Quem fez isso com você,Quem fez isso com você.1
0,2.0,1.0,1.0,3.0,15.0,2.0,1.0,2.0,1.0,1.0,...,55.0,11,1,1,1.0,1.0,5.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.0,11,1,1,4.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.0,11,1,1,2.0,NaN,10.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,11,1,1,2.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,11,1,1,4.0,NaN,NaN,NaN,NaN,NaN
